In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
from datetime import datetime
import folium
import scipy
import json
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 5]
pd.set_option('display.float_format', lambda x:'%f'%x)
from IPython.display import display
pd.options.display.max_columns = None
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder

# EDA

In this challenge, you are given a list of users along with their demographics, web session records, and some summary statistics. You are asked to predict which country a new user's first booking destination will be. All the users in this dataset are from the USA.

There are 12 possible outcomes of the destination country: 'US', 'FR', 'CA', 'GB', 'ES', 'IT', 'PT', 'NL','DE', 'AU', 'NDF' (no destination found), and 'other'. Please note that 'NDF' is different from 'other' because 'other' means there was a booking, but is to a country not included in the list, while 'NDF' means there wasn't a booking.

The training and test sets are split by dates. In the test set, you will predict all the new users with first activities after 7/1/2014 (note: this is updated on 12/5/15 when the competition restarted). In the sessions dataset, the data only dates back to 1/1/2014, while the users dataset dates back to 2010. 


**File descriptions**
- **train_users.**csv - the training set of users


- **test_users.csv** - the test set of users
    - id: user id
    - date_account_created: the date of account creation
    - timestamp_first_active: timestamp of the first activity, note that it can be earlier than date_account_created or date_first_booking because a user can search before signing up
    - date_first_booking: date of first booking
    - gender
    - age
    - signup_method
    - signup_flow: the page a user came to signup up from
    - language: international language preference
    - affiliate_channel: what kind of paid marketing
    - affiliate_provider: where the marketing is e.g. google, craigslist, other
    - first_affiliate_tracked: whats the first marketing the user interacted with before the signing up
    - signup_app
    - first_device_type
    - first_browser
    - country_destination: this is the target variable you are to predict


- **sessions.csv** - web sessions log for users
    - user_id: to be joined with the column 'id' in users table
    - action
    - action_type
    - action_detail
    - device_type
    - secs_elapsed


- **countries.csv** - summary statistics of destination countries in this dataset and their locations


- **age_gender_bkts.csv** - summary statistics of users' age group, gender, country of destination


- **sample_submission.csv** - correct format for submitting your predictions

In [3]:
age_gender_bkts = pd.read_csv('../data/airbnb/age_gender_bkts.csv')
age_gender_bkts[:5]

,age_bucket,country_destination,gender,population_in_thousands,year
0,100+,AU,male,1.000000,2015.000000
1,95-99,AU,male,9.000000,2015.000000
2,90-94,AU,male,47.000000,2015.000000
3,85-89,AU,male,118.000000,2015.000000
4,80-84,AU,male,199.000000,2015.000000


In [8]:
age_gender_bkts.shape

(420, 5)

In [11]:
countries = pd.read_csv('../data/airbnb/countries.csv')
countries[:11]

,country_destination,lat_destination,lng_destination,distance_km,destination_km2,destination_language,language_levenshtein_distance
0,AU,-26.853388,133.275160,15297.744000,7741220.000000,eng,0.000000
1,CA,62.393303,-96.818146,2828.133300,9984670.000000,eng,0.000000
2,DE,51.165707,10.452764,7879.568000,357022.000000,deu,72.610000
3,ES,39.896027,-2.487694,7730.724000,505370.000000,spa,92.250000
4,FR,46.232193,2.209667,7682.945000,643801.000000,fra,92.060000
5,GB,54.633220,-3.432277,6883.659000,243610.000000,eng,0.000000
6,IT,41.873990,12.564167,8636.631000,301340.000000,ita,89.400000
7,NL,52.133057,5.295250,7524.320300,41543.000000,nld,63.220000
8,PT,39.553444,-7.839319,7355.253400,92090.000000,por,95.450000
9,US,36.966427,-95.844030,0.000000,9826675.000000,eng,0.000000


In [9]:
countries.shape

(10, 7)

In [5]:
sessions = pd.read_csv('../data/airbnb/sessions.csv')
sessions[:5]

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.000000
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.000000
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.000000
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.000000
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.000000


In [14]:
sessions.shape

(10567737, 6)

In [17]:
sessions.action.value_counts()[:20]

show                     2768278
index                     843699
search_results            725226
personalize               706824
search                    536057
ajax_refresh_subtotal     487744
update                    365130
similar_listings          364624
social_connections        339000
reviews                   320591
active                    188036
similar_listings_v2       168788
lookup                    162041
create                    155887
dashboard                 152952
header_userpic            141830
collections               124417
edit                      109083
campaigns                 105028
track_page_view            81117
Name: action, dtype: int64

In [7]:
train_users = pd.read_csv('../data/airbnb/train_users.csv')
train_users[:5]

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,nan,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.000000,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.000000,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.000000,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.000000,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [18]:
train_users.shape

(213451, 16)

In [6]:
test_users = pd.read_csv('../data/airbnb/test_users.csv')
test_users[:5]

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser
0,5uwns89zht,2014-07-01,20140701000006,nan,FEMALE,35.000000,facebook,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
1,jtl0dijy2j,2014-07-01,20140701000051,nan,-unknown-,nan,basic,0,en,direct,direct,untracked,Moweb,iPhone,Mobile Safari
2,xx0ulgorjt,2014-07-01,20140701000148,nan,-unknown-,nan,basic,0,en,direct,direct,linked,Web,Windows Desktop,Chrome
3,6c6puo6ix0,2014-07-01,20140701000215,nan,-unknown-,nan,basic,0,en,direct,direct,linked,Web,Windows Desktop,IE
4,czqhjk3yfe,2014-07-01,20140701000305,nan,-unknown-,nan,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Safari


In [12]:
test_users.shape

(62096, 15)